In [8]:
import numpy as np
from matplotlib import pyplot as plt
from dsapol import dsapol
from scipy.signal import correlate
from scipy.signal import savgol_filter as sf
from scipy.signal import convolve
from scipy.signal import fftconvolve
from scipy.ndimage import convolve1d
from scipy.signal import peak_widths
from scipy.stats import chi
from scipy.stats import norm
import panel as pn
pn.extension()
#from numpy.ma import masked_array as ma
import numpy.ma as ma
from scipy.stats import kstest
from scipy.optimize import curve_fit
import time
#import numpy.ma as ma 
%load_ext autoreload
%autoreload 2 

from scipy.signal import find_peaks
from scipy.signal import peak_widths
import copy
import numpy as np
import os
import param

from sigpyproc import FilReader
from sigpyproc.Filterbank import FilterbankBlock
from sigpyproc.Header import Header
from matplotlib import pyplot as plt
import pylab
import pickle
import json
from scipy.interpolate import interp1d
from scipy.stats import chi2
from scipy.stats import chi
from scipy.signal import savgol_filter as sf
from scipy.signal import convolve
from scipy.ndimage import convolve1d
from RMtools_1D.do_RMsynth_1D import run_rmsynth
from RMtools_1D.do_RMclean_1D import run_rmclean
from RMtools_1D.do_QUfit_1D_mnest import run_qufit

plt.rcParams.update({
                    'font.size': 16,
                    'font.family': 'serif',
                    'axes.labelsize': 16,
                    'axes.titlesize': 16,
                    'xtick.labelsize': 12,
                    'ytick.labelsize': 12,
                    'xtick.direction': 'in',
                    'ytick.direction': 'in',
                    'xtick.top': True,
                    'ytick.right': True,
                    'lines.linewidth': 1,
                    'lines.markersize': 5,
                    'legend.fontsize': 14,
                    'legend.borderaxespad': 0,
                    'legend.frameon': False,
                    'legend.loc': 'lower right'})


from astropy.time import Time
from astropy.coordinates import EarthLocation
import astropy.units as u


def gauss_scint(x,bw,amp,off):
    return off + amp*np.exp(-np.log(2)*((x/bw)**2))


def lorentz_scint(x,bw,amp,off):
    return off + amp*(bw/(x**2 + (0.5*bw**2)))

fsize=30
fsize2=20
plt.rcParams.update({
                    'font.size': fsize,
                    'font.family': 'sans-serif',
                    'axes.labelsize': fsize,
                    'axes.titlesize': fsize,
                    'xtick.labelsize': fsize,
                    'ytick.labelsize': fsize,
                    'xtick.direction': 'in',
                    'ytick.direction': 'in',
                    'xtick.top': True,
                    'ytick.right': True,
                    'lines.linewidth': 1,
                    'lines.markersize': 5,
                    'legend.fontsize': fsize2,
                    'legend.borderaxespad': 0,
                    'legend.frameon': False,
                    'legend.loc': 'lower right'})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from pol_panel_funcs import pol_plot
from pol_panel_funcs import pol_panel
from pol_panel_funcs import RM_plot
from pol_panel_funcs import RM_panel


# Try again, implementing based on Casey's Template

In [57]:



def build_pol_tab():
    
    
    
    # make dropdown box to choose FRB file
    x=os.listdir("/media/ubuntu/ssd/sherman/scratch_weights_update_2022-06-03_32-7us") 
    FRB_list = []
    for i in range(len(x)):
        if len(x[i]) >= 10 and x[i][10] == "_":
            FRB_list.append(x[i])

    
    
    #(I_t_init,Q_t_init,U_t_init,V_t_init) = dsapol.get_stokes_vs_time(I,Q,U,V,1,fobj.header.tsamp,n_t,n_off=int(12000//n_t),plot=True,show=True,normalize=True,buff=1,window=30)
    menu_calplots = pn.widgets.MenuButton(name="Select FRB to Load",
                                          items=FRB_list,
                                         width=200)
    
    load_button = pn.widgets.Button(name='Load FRB', width=100, button_type='primary')
    
    # make dropdown box to choose calibration file
    y = os.listdir("/media/ubuntu/ssd/sherman/code")
    CAL_list = []
    
    for i in range(len(y)):
        if ("POLCAL_PARAMETERS" in y[i]) and ("csv" in y[i]):
            CAL_list.append(y[i])
    calmenu_calplots = pn.widgets.MenuButton(name="Select Calibration File to Load",
                                      items=CAL_list,
                                     width=200)
    
    cal_button = pn.widgets.Button(name='Calibrate', width=100, button_type='primary')
    ibeam_input = pn.widgets.IntSlider(name='ibeam', start=0, end=250, step=1, value=0)
    RA_input = pn.widgets.FloatInput(name='RA (degrees)', value=0, step=1e-2, start=0, end=360)
    DEC_input = pn.widgets.FloatInput(name='DEC (degrees)', value=0, step=1e-2, start=-90, end=90)
    
    def callback(target, event):
        target.error = "Loading FRB..."
        #target.frb_submitted = True

    def callback_cal(target,event):
        target.error = "Calibrating FRB..."
        
    pane_test = pn.pane.Markdown('# DSA-110 Polarization Dashboard')
    pan1 = pol_panel()
    
    menu_calplots.link(pan1, clicked='frb_name')
    load_button.link(pan1,callbacks={'clicks': callback})#pan1, clicks='frb_submitted')
    
    calmenu_calplots.link(pan1,clicked='cal_name')
    cal_button.link(pan1,callbacks={'clicks': callback_cal})#pan1, clicks='frb_submitted')
    ibeam_input.link(pan1,value='ibeam')
    RA_input.link(pan1,value='RA')
    DEC_input.link(pan1,value='DEC')
    
    pan2 = RM_panel()
    #link IQUV from panel1 to panel 2
    def callback_link(target,event):
        target.error = "Transferring data between panels..."
        t1 = time.time()
        
        
        target.I = pan1.I
        target.Q = pan1.Q
        target.U = pan1.U
        target.V = pan1.V
        
        #get current component, or if all already done, get full thing
        if (len(pan1.comp_dict.keys()) <= len(pan1.fixed_comps)) and (len(pan1.comp_dict.keys()) > 0) and pan1.filt_weights_on:
            # get current component
            target.curr_comp = np.max(list(pan1.comp_dict.keys()))
            target.error = "Multiple Components, using component " + str(target.curr_comp) + " " + str(len(pan1.comp_dict[target.curr_comp]["I_f_init"]))
                                                                                                       
            
            target.I_f = pan1.comp_dict[target.curr_comp]["I_f_init"]#pan1.I_f_init
            target.Q_f = pan1.comp_dict[target.curr_comp]["Q_f_init"]#pan1.Q_f_init
            target.U_f = pan1.comp_dict[target.curr_comp]["U_f_init"]#pan1.U_f_init
            target.V_f = pan1.comp_dict[target.curr_comp]["V_f_init"]#pan1.V_f_init
        
            target.curr_weights = pan1.comp_dict[target.curr_comp]["weights"]
            target.ibox = pan1.comp_dict[target.curr_comp]["ibox"]
        elif (len(pan1.comp_dict.keys()) == len(pan1.fixed_comps)) and (not pan1.filt_weights_on):
            #get full component
            target.curr_comp = -1
            target.error = "All Components " + str(len(pan1.I_f_init))
            
            target.I_f = pan1.I_f_init
            target.Q_f = pan1.Q_f_init
            target.U_f = pan1.U_f_init
            target.V_f = pan1.V_f_init
            
            target.curr_weights = pan1.curr_weights
            target.ibox = pan1.ibox
        target.freq_test = pan1.freq_test_init
        target.n_t = pan1.n_t
        target.n_f = pan1.n_f
        #target.timestart = pan1.timestart
        #/itarget.timestop = pan1.timestop
        target.fobj = pan1.fobj
        target.comp_dict = pan1.comp_dict
        
        #reset flags
        target.init_RM = True
        target.fine_RM = False
        
        #target.error = "Complete: " + str(np.around(time.time()-t1,2)) + " s to transfer data"
        
    def callback_linkback(target,event):
        target.comp_dict = pan2.comp_dict
        
    link_button = pn.widgets.Button(name='Proceed to RM Synthesis',width=200,button_type='primary')
    link_button.link(pan2,callbacks={'clicks': callback_link})#pan1, clicks='frb_submitted')
    
    linkback_button = pn.widgets.Button(name='Return to Pol Analysis',width=200,button_type='primary')
    linkback_button.link(pan1,callbacks={'clicks':callback_linkback})
    
    
    #RM calibration
    def callback_RMcal(target,event):
        #get most recently estimated RM
        
        try:
            if not pan2.fine_RM:
                rmcal = pan2.RM1
            else:
                rmcal = pan2.RM2zoom

            pan2.error = str(pan2.curr_comp)
            #Case 1: calibrate full burst
            if pan2.curr_comp == -1:
                target.error = "Derotating full burst to RM = " + str(np.around(rmcal,2)) + " rad/m^2..."
                t1 = time.time()
                I_RMcal_init,Q_RMcal_init,U_RMcal_init,V_RMcal_init = dsapol.calibrate_RM(target.I_init,target.Q_init,target.U_init,target.V_init,rmcal,0,target.freq_test,stokes=True)
                target.I_RMcal,target.Q_RMcal,target.U_RMcal,target.V_RMcal = dsapol.calibrate_RM(target.I,target.Q,target.U,target.V,rmcal,0,target.freq_test,stokes=True)

                (target.I_f_init,target.Q_f_init,target.U_f_init,target.V_f_init) = dsapol.get_stokes_vs_freq(target.I_RMcal,target.Q_RMcal,target.U_RMcal,target.V_RMcal,1,target.fobj.header.tsamp,1,target.n_t,target.freq_test_init,n_off=int(12000/target.n_t),plot=False,show=False,normalize=True,weighted=True,timeaxis=target.timeaxis,fobj=target.fobj,input_weights=target.curr_weights)

                (target.I_t_init,target.Q_t_init,target.U_t_init,target.V_t_init) = dsapol.get_stokes_vs_time(I_RMcal_init,Q_RMcal_init,U_RMcal_init,V_RMcal_init,1,target.fobj.header.tsamp,1,n_off=int(12000/1),plot=False,show=False,normalize=True)


                target.error = "Complete: " + str(np.around(time.time()-t1,2)) + " to derotate"

                #multipeaks_all = (len(target.fixed_comps) > 1) or (target.comp_dict[0]["multipeaks"])
                #[(pol_f,pol_t,avg_frac,sigma_frac,snr_frac),(L_f,L_t,avg_L,sigma_L,snr_L),(C_f,C_t,avg_C_abs,sigma_C_abs,snr_C),(C_f,C_t,avg_C,sigma_C,snr_C),snr] = dsapol.get_pol_fraction(target.I_RMcal,target.Q_RMcal,target.U_RMcal,target.V_RMcal,target.ibox,target.fobj.header.tsamp,target.n_t,1,target.freq_test_init,n_off=int(12000/target.n_t),normalize=True,weighted=True,timeaxis=target.timeaxis,fobj=target.fobj,multipeaks=multipeaks_all,height=target.height*np.max(target.curr_weights)/np.max(target.I_t),input_weights=target.curr_weights)

                
            #Case 2: calibrate current component
            elif pan2.curr_comp != -1:
                target.error = "Derotating component " + str(pan2.curr_comp) + " to RM = " + str(np.around(rmcal,2)) + " rad/m^2..."
                t1 = time.time()
                I_RMcal_init,Q_RMcal_init,U_RMcal_init,V_RMcal_init = dsapol.calibrate_RM(target.I_init,target.Q_init,target.U_init,target.V_init,rmcal,0,target.freq_test,stokes=True)
                target.I_RMcal,target.Q_RMcal,target.U_RMcal,target.V_RMcal = dsapol.calibrate_RM(target.I,target.Q,target.U,target.V,rmcal,0,target.freq_test,stokes=True)

                #(target.I_f_init,target.Q_f_init,target.U_f_init,target.V_f_init) = dsapol.get_stokes_vs_freq(target.I_RMcal,target.Q_RMcal,target.U_RMcal,target.V_RMcal,1,target.fobj.header.tsamp,1,target.n_t,target.freq_test_init,n_off=int(12000/target.n_t),plot=False,show=False,normalize=True,weighted=True,timeaxis=target.timeaxis,fobj=target.fobj,input_weights=target.curr_weights)
                target.comp_dict[pan2.curr_comp]["I_f_init"], target.comp_dict[pan2.curr_comp]["Q_f_init"], target.comp_dict[pan2.curr_comp]["U_f_init"], target.comp_dict[pan2.curr_comp]["V_f_init"] = dsapol.get_stokes_vs_freq(target.I_RMcal,target.Q_RMcal,target.U_RMcal,target.V_RMcal,1,target.fobj.header.tsamp,1,target.n_t,target.freq_test_init,n_off=int(12000/target.n_t),plot=False,show=False,normalize=True,weighted=True,timeaxis=target.timeaxis,fobj=target.fobj,input_weights=target.comp_dict[pan2.curr_comp]["weights"])
                target.comp_dict[pan2.curr_comp]["I_f"], target.comp_dict[pan2.curr_comp]["Q_f"], target.comp_dict[pan2.curr_comp]["U_f"], target.comp_dict[pan2.curr_comp]["V_f"] = target.comp_dict[pan2.curr_comp]["I_f_init"], target.comp_dict[pan2.curr_comp]["Q_f_init"], target.comp_dict[pan2.curr_comp]["U_f_init"], target.comp_dict[pan2.curr_comp]["V_f_init"]
                
                #(target.I_t_init,target.Q_t_init,target.U_t_init,target.V_t_init) = dsapol.get_stokes_vs_time(I_RMcal_init,Q_RMcal_init,U_RMcal_init,V_RMcal_init,1,target.fobj.header.tsamp,1,n_off=int(12000/1),plot=False,show=False,normalize=True)

                target.error = "Complete: " + str(np.around(time.time()-t1,2)) + " to derotate"

                #multipeaks_all = (len(target.fixed_comps) > 1) or (target.comp_dict[0]["multipeaks"])
                #[(pol_f,pol_t,avg_frac,sigma_frac,snr_frac),(L_f,L_t,avg_L,sigma_L,snr_L),(C_f,C_t,avg_C_abs,sigma_C_abs,snr_C),(C_f,C_t,avg_C,sigma_C,snr_C),snr] = dsapol.get_pol_fraction(target.I_RMcal,target.Q_RMcal,target.U_RMcal,target.V_RMcal,target.ibox,target.fobj.header.tsamp,target.n_t,1,target.freq_test_init,n_off=int(12000/target.n_t),normalize=True,weighted=True,timeaxis=target.timeaxis,fobj=target.fobj,multipeaks=multipeaks_all,height=target.height*np.max(target.curr_weights)/np.max(target.I_t),input_weights=target.curr_weights)

                                

            
        except Exception as e:
            target.error2 = "From callback_RMcal(): " + str(e)
    
    
    RMcal_button = pn.widgets.Button(name='RM Calibrate',width=200,button_type='primary')
    RMcal_button.link(pan1,callbacks={'clicks':callback_RMcal})
    
    
    
    return pn.Column(pane_test,pn.Row(menu_calplots,load_button,calmenu_calplots,ibeam_input,RA_input,DEC_input,cal_button,link_button,linkback_button,RMcal_button),pn.Row(pan1.param,pan1.view,pan2.param,pan2.view))


#ADD JAKOB'S CODE HERE FOR BURSTFIT TAB
def burstfit_tab():
    pass
    
    
#usage tab

def build_usage_tab():
    
    #dedisp tab
    dedisp_usage = """
    ### Dedispersion Tab Usage
    
    Insert dedispersion tab usage here
    """
    
    #pol tab
    pol_usage = """
    ### Polarization Tab Usage
    
    Insert pol tab usage here
    """
    
    #burstfit tab
    burstfit_usage = """
    ### Burstfit Tab Usage
    
    Insert burstfit tab usage here
    """
    
    #
    width=1000
    
    return pn.Column(pn.pane.Markdown("# Usage: This page contains instructions and examples for using the Burst Analysis Dashboard",width=width),
                     pn.pane.Markdown(dedisp_usage,width=width),
                    pn.pane.Markdown(pol_usage,width=width),
                    pn.pane.Markdown(burstfit_usage,width=width))

In [58]:
pol_pane = pn.param.ParamFunction(build_pol_tab,lazy=True)


In [59]:
usage_pane = pn.param.ParamFunction(build_usage_tab,lazy=True)

In [60]:
dashboard = pn.Column(pn.Row(pn.layout.tabs.Tabs(('Usage', usage_pane),
                                                 ('Polarization', pol_pane),
                                                 #('Burstfit',burstfit_tab),
                                                dynamic=True, active=0),
                             margin=(10, 10, 10, 10), background='#f0f0f0'))

In [61]:
dashboard

Column
    [0] Row(background='#f0f0f0', margin=(10, 10, 10, 10))
        [0] Tabs(dynamic=True)
            [0] ParamFunction(function, lazy=True)
            [1] ParamFunction(function, lazy=True)

In [62]:
dashboard.servable(title="Burst Analysis Dashboard")

Column
    [0] Row(background='#f0f0f0', margin=(10, 10, 10, 10))
        [0] Tabs(dynamic=True)
            [0] ParamFunction(function, lazy=True)
            [1] ParamFunction(function, lazy=True)